In [2]:
# setting up selenium/ beautifulsoup/ pandas
import pandas 
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC



def launchBrowser():
    options = webdriver.ChromeOptions()
    options.add_experimental_option("detach", True)
    service = Service(executable_path="chromedriver.exe")
    driver = webdriver.Chrome(service=service, options=options)

    driver.get("https://edition.cnn.com/election/2024/primaries-and-caucuses/results/alabama")



launchBrowser()


In [ ]:
# set up launchbrowser function 
# create list to store data for each page  
# "for" loop to loop over pages 
# convert to pandas dataframe